<a href="https://colab.research.google.com/github/tukamilano/anagram/blob/main/anagram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fugashi unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 43.2 MB/s eta 0:00:00
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=4d6db14220d72a730d67a03eaa3675d003d88c8f460cd8f460bba711f8fa7fca
  Stored in directory: /root/.cache/pip/wheels/5e/1f/0f/4d43887e5476d956fae828ee9b6687becd5544d68b51ed633d
Successfully built unidic-lite


In [2]:
import shutil
import os
from google.colab import drive

drive.mount('/content/drive')

# Google Driveのファイルパスを指定
file_path = "/content/drive/MyDrive/anagram/model.bin"

# コピー先のファイル名を指定
destination_file_name = "model.bin"

# Google Driveがマウントされているか確認し、ファイルをコピー
if os.path.exists(file_path):
    shutil.copy(file_path, destination_file_name)
    print(f"'{file_path}' を '{destination_file_name}' にコピーしました。")
else:
    print(f"エラー: ファイル '{file_path}' が見つかりません。Google Driveがマウントされているか、パスが正しいか確認してください。")
    print("Google Driveをマウントするには、次のコードを実行してください: from google.colab import drive; drive.mount('/content/drive')")

Mounted at /content/drive
'/content/drive/MyDrive/anagram/model.bin' を 'model.bin' にコピーしました。


In [3]:
import os

# Clone KenLM if not already cloned
if not os.path.exists("kenlm"):
    !git clone https://github.com/kpu/kenlm.git
    print("KenLM repository cloned.")
else:
    print("KenLM repository already exists.")

# Build KenLM
# Change to kenlm directory
%cd kenlm

# Create build directory and navigate into it
!mkdir -p build
%cd build

# Configure and build KenLM
!cmake ..
!make -j $(nproc)

# Navigate back to the original content directory
%cd /content

print("KenLM build process complete. You can now try to run the lmplz command.")

Cloning into 'kenlm'...
remote: Enumerating objects: 14185, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 14185 (delta 117), reused 81 (delta 81), pack-reused 14010 (from 2)
Receiving objects: 100% (14185/14185), 5.91 MiB | 11.23 MiB/s, done.
Resolving deltas: 100% (8059/8059), done.
KenLM repository cloned.
/content/kenlm
/content/kenlm/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
CMake Warning (dev) at CMakeLis

In [4]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     | 553.6 kB 2.5 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp312-cp312-linux_x86_64.whl size=3188003 sha256=70721b8d7ee7f844880b295e2d2be11a44f5fa6b8fc669ceb5cfbbc795051ed7
  Stored in directory: /tmp/pip-ephem-wheel-cache-5bkox139/wheels/92/c8/12/56d187154e078f0eaa74d059017fc1afe1c4d91fbce02ce8d9
Successfully built kenlm


In [5]:
import kenlm

model = kenlm.Model("model.bin")

In [6]:
import fugashi

# ==== 初期化 ====
print("形態素解析器を初期化中...")
tagger = fugashi.Tagger()

形態素解析器を初期化中...


In [24]:
import random
import math
from itertools import combinations

# ===============
# ひらがなセット
# ===============
jp = list("あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわをん")

# ===============================
# Perplexityを計算する関数（本物）
# ===============================
def calculate_perplexity(sentence):
    """
    KenLM を使って
    与えられた文の Perplexity（自然さ）を算出する。
    sentence: もしくは str
    """
    sentence = ''.join(sentence)
    processed_sentence_list = [word.surface for word in tagger(sentence)]
    processed_sentence = ' '.join(processed_sentence_list)
    ppl = model.perplexity(processed_sentence)
    ppl += 1000 * len(processed_sentence_list)
    return ppl

In [25]:
def improve_by_4_opt(sentence, step=1000000):
    v = calculate_perplexity(sentence)
    calculate_num = 0
    improved = True

    while improved:
        improved = False
        indices = list(range(len(sentence)))
        base = list(combinations(indices, 4))
        random.shuffle(base)  # 組の順序をランダム化

        for four_var in base:
            a, b, c, d = four_var
            new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
            new_v = calculate_perplexity(new_sentence)
            calculate_num += 1

            if new_v < v:
                v = new_v
                sentence = new_sentence
                improved = True
                print(f"Improved! step={calculate_num}, new_ppl={v:.3f}")
                break

            if calculate_num >= step:
                print(f"Reached step limit ({step}). Stop.")
                return sentence

    print("No more improvement.")
    return sentence

def improve_by_4_opt_sa(sentence, step=1_000_000, T0=1000.0, Tmin=0.1, alpha=0.99999):
    v = calculate_perplexity(sentence)
    best_sentence, best_v = sentence, v
    T = T0

    for step_num in range(step):
        # ランダムに4点選択
        indices = sorted(random.sample(range(len(sentence)), 4))
        a, b, c, d = indices
        new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
        new_v = calculate_perplexity(new_sentence)
        delta = new_v - v

        # 改善 or 確率的悪化許容
        if delta < 0 or random.random() < math.exp(-delta / T):
            sentence, v = new_sentence, new_v
            if v < best_v:
                best_sentence, best_v = sentence, v
                print(f"Step {step_num}: Improved! new_ppl={v:.3f}")

        # 温度を減衰
        T = max(T * alpha, Tmin)

        # ログ出力（確認用）
        if step_num % 1000 == 0:
            print(f"Step {step_num}: T={T:.3f}, ppl={v:.3f}")

    print(f"Finished all {step} steps.")
    print(f"Best perplexity: {best_v:.3f}")
    return best_sentence


# ========================
# 実行部分
# ========================
def main():
    sentence = jp[:]  # コピーしておく
    random.shuffle(sentence)
    print("Initial sentence:", "".join(sentence))
    improved_sentence = improve_by_4_opt_sa(sentence)
    print("Final sentence:", "".join(improved_sentence))

if __name__ == "__main__":
    main()


Initial sentence: ろみれつえなをちゆきこけとさしひりてにくんあぬへやそめるはませむすのういらふわよおもかねたほ
Step 0: Improved! new_ppl=35311.210
Step 0: T=999.990, ppl=35311.210
Step 3: Improved! new_ppl=34574.194
Step 11: Improved! new_ppl=34040.056
Step 13: Improved! new_ppl=33897.365
Step 25: Improved! new_ppl=33828.341
Step 47: Improved! new_ppl=33120.430
Step 49: Improved! new_ppl=33026.636
Step 51: Improved! new_ppl=32890.325
Step 52: Improved! new_ppl=32345.755
Step 146: Improved! new_ppl=32224.666
Step 147: Improved! new_ppl=32018.519
Step 151: Improved! new_ppl=31988.246
Step 153: Improved! new_ppl=31906.180
Step 154: Improved! new_ppl=31622.991
Step 166: Improved! new_ppl=31344.421
Step 586: Improved! new_ppl=30675.937
Step 587: Improved! new_ppl=30455.906
Step 623: Improved! new_ppl=30256.164
Step 1000: T=990.040, ppl=35011.132
Step 2000: T=980.189, ppl=35223.324
Step 3000: T=970.436, ppl=37122.488
Step 3444: Improved! new_ppl=30157.327
Step 4000: T=960.780, ppl=34345.579
Step 5000: T=951.220, ppl=34687.547
Step 6000:

In [13]:
text = "ひしめくほんのもりをつむみなとゆれるふねはおきへぬけたてやよこにそろうまちせかいさえあらわす"
processed_sentence = ' '.join(word.surface for word in tagger(text))
print(processed_sentence)

5365.465848041176
